In [46]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle

In [13]:
# Step 1: Prepare the data
# Assuming you have a dataset with columns 'Question', 'Answer', 'Marks'
data = pd.read_csv('lecture_data.csv')
data.head()

Question                                             Answer  \
0  what is a computer?  A computer is a device that accepts informatio...   
1  what is a computer?  A computer is a tool used for word processing,...   
2  what is a computer?  A computer is a machine used for calculations ...   
3  what is a computer?  A computer is a device that can store and retr...   
4  what is a computer?  A computer is a machine that can communicate w...   

   Marks  
0     10  
1      5  
2      7  
3      7  
4      8

In [14]:
data.dropna(inplace=True) 

In [47]:
# Step 3: Text Cleaning
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation (replace with empty space)
    text = text.replace('[^\w\s]', '')
    
    # Remove stopwords and lemmatize
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    # Join tokens back into a single string
    processed_text = ' '.join(filtered_tokens)
    
    return processed_text

In [54]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [55]:
data['Question_Answer'] = data['Question'] + ' ' + data['Answer']
data['Cleaned_Text'] = data['Question_Answer'].apply(preprocess_text)

In [56]:
data = shuffle(data, random_state=42)


In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    data['Cleaned_Text'],
    data['Marks'],
    test_size=0.2,
    random_state=42
)

In [58]:
def synonym_replacement(text, num_replacements=3):
    words = nltk.word_tokenize(text)
    for i in range(num_replacements):
        for j in range(len(words)):
            syns = wordnet.synsets(words[j])
            if syns:
                words[j] = syns[0].lemmas()[0].name()
    augmented_text = ' '.join(words)
    return augmented_text


In [60]:
X_train_augmented = X_train.apply(lambda x: synonym_replacement(x, num_replacements=3))
y_train_augmented = y_train.copy()
X_train = pd.concat([X_train, X_train_augmented], ignore_index=True)
y_train = pd.concat([y_train, y_train_augmented], ignore_index=True)

In [61]:
# Step 5: Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [62]:
# Step 6: Oversample the training data to address class imbalance
oversampler = RandomOverSampler(random_state=42)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train_vectorized, y_train)

In [63]:
# Step 7: Train the model (Random Forest Classifier)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_oversampled, y_train_oversampled)

RandomForestClassifier(random_state=42)

In [45]:
# Step 6: Train the model
svm_model = SVC()
svm_model.fit(X_train_vectorized, y_train)

SVC()

In [64]:
# Step 7: Evaluate the model (optional)
y_pred = rf_model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy}")

Model accuracy: 0.3333333333333333


In [65]:
# Step 8: Use the trained model for new student answers
def analyze_answers(new_answers):
    cleaned_answers = [preprocess_text(answer) for answer in new_answers]
    new_X = vectorizer.transform(cleaned_answers)
    predicted_marks = rf_model.predict(new_X)
    return predicted_marks

In [72]:
new_student_answers = [
    "What is computer?: SLIIT",
]
predicted_marks = analyze_answers(new_student_answers)
print(f"Predicted marks: {predicted_marks}")

Predicted marks: [6]
